In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

from settings import *
from lo_step import lo_step

### Load sample data
The sample data has a time interval of 1 second

In [2]:
path_input = os.path.join("data", "sample_data.csv")
path_output = os.path.join("results", f"load_with_BESS.csv")
data = pd.read_csv(path_input)
data.head()

,Unnamed: 0,datetime,solar_load,building_load
0,513189,20220531-000000,-0.0,22.838876
1,513190,20220531-000001,-0.0,24.012708
2,513191,20220531-000002,-0.0,23.201554
3,513192,20220531-000003,-0.0,22.433628
4,513193,20220531-000004,-0.0,22.718627


### Extract time series of solar power and building load

In [7]:
dts = np.array(data['datetime'])
solar_loads = np.array(data['solar_load'])
building_loads = np.array(data['building_load'])
N = len(dts) # Total time steps

### Initialize parameters

In [8]:
Ppeak = 0 # Initialize peak power
SoC = 0.5 # Initialize SoC of the BESS
SoCs = [SoC] # Initialize SoC time series
BESS_outputs = [] # Initialize BESS time series
Qs = []

### Simulation

In [9]:
for i in range(N):
    dt = datetime.strptime(dts[i], "%Y%m%d-%H%M%S")
    
    # Determine weekday
    if dt.weekday() in [0, 1, 2, 3, 4]:
        # Determine month
        if dt.month in [3, 4]:
            peakHourTable = peakHourTable_weekdayMarApr
        else:
            peakHourTable = peakHourTable_weekday
    else:
        peakHourTable = peakHourTable_weekend
    
    # Determine season
    if dt.month in summerMonths:
        energyChargeRef = energyChargeRef_Summer
        demandChargeRef = demandChargeRef_Summer
    else:
        energyChargeRef = energyChargeRef_Winter
        demandChargeRef = demandChargeRef_Winter
    
    # Get energy charge and demand charge at t
    peak_indicator = peakHourTable[dt.hour]
    pE = energyChargeRef[peak_indicator]
    pF = pE # net-metering
    pD = demandChargeRef[peak_indicator]
    V = VsTable[peak_indicator]
    
    # Get BESS output
    LB = building_loads[i]
    PS = solar_loads[i]
    x = lo_step(LB, PS, Ppeak, SoC, pE, pF, pD, V, DT)
    BESS_outputs.append(x)
    
    # Update SoC and Ppeak
    E = SoC*E_FULL - x*DT - abs(x)*DT*(1-sqrt(K))
    SoC = E/E_FULL
    SoCs.append(SoC)
    Ppeak = max(Ppeak, LB-PS-x)
    
    Q = E_MAX - SoC*E_FULL - DT*P_MAX*sqrt(K)
    Qs.append(Q)
    
    if i % 10000 == 0:
        print(i, dt)
print("Finished")

0 2022-05-31 00:00:00
10000 2022-05-31 02:47:52
20000 2022-05-31 05:35:35
30000 2022-05-31 08:23:16
40000 2022-05-31 11:11:10
50000 2022-05-31 13:59:04
60000 2022-05-31 16:46:50
70000 2022-05-31 19:34:45
80000 2022-05-31 22:22:26
90000 2022-06-01 01:28:03
100000 2022-06-01 04:15:55
110000 2022-06-01 07:03:47
120000 2022-06-01 09:51:35
130000 2022-06-01 12:39:22
140000 2022-06-01 15:27:11
150000 2022-06-01 18:15:08
160000 2022-06-01 21:03:01
170000 2022-06-01 23:50:57
Finished


### Output results

In [10]:
info_export = {'datetime': dts,\
               'solar_load': solar_loads,\
               'building_load': building_loads,\
               'BESS_output': BESS_outputs}
df = pd.DataFrame(info_export)
df.to_csv(path_output)